In [ ]:
#!pip install clustergram pandas_profiling scipy sklearn statsmodels IPython dtale matplotlib rpy2 seaborn shap

In [ ]:
#put in ~/.bashrc
#LD_PRELOAD="/mnt/distvol/R/4.1.2/lib64/R/lib/LibR.so"

In [ ]:
#from fracdiff import fdiff
#import urbangrammar-graphics as ugg
import os
from clustergram import Clustergram
from concurrent.futures import wait, ALL_COMPLETED
from dask.distributed import Client
from dask.distributed import as_completed
from numpy import absolute
from numpy import arange
from numpy import mean
from numpy import std
from pandas import read_csv
from pandas_profiling import ProfileReport
#from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
from rpy2.robjects.packages import importr
from rpy2.rinterface_lib import openrlib
from scipy import stats
from scipy.cluster.vq import vq
from scipy.spatial.distance import cdist, pdist
from scipy.special import boxcox, inv_boxcox
from scipy.stats import f
from sklearn import preprocessing
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import *
#from sklearn.preprocessing import PowerTransformer
#from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing import scale
from sklearn.utils import as_float_array
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.stats.outliers_influence import OLSInfluence
import IPython
import concurrent.futures
import dask.dataframe as dd
import datetime
import dtale
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pingouin as pg
import re
import rpy2
import rpy2.robjects as ro
import rpy2.situation
import scipy
import seaborn as sns
import shap
import sklearn
import sklearn.linear_model
import statsmodels.api as sm
import statsmodels.tools
import sys
import time

In [ ]:
#c = get_config()
libpath = os.environ.get('LD_LIBRARY_PATH', '')
os.environ['LD_LIBRARY_PATH'] = (
    rpy2.situation.r_ld_library_path_from_subprocess(openrlib.R_HOME) +
    libpath
)

In [286]:
def testNormal (x):    
    
    k2, p = stats.normaltest(x)
    alpha = .001
    #print("p = {:g}".format(p))    
    if p < alpha:  # null hypothesis: x comes from a normal distribution
        #print(p)
        #print(alpha)
        print("The null hypothesis can be rejected")
        xt, _ = stats.yeojohnson(x)
        #xt, _ = stats.boxcox(x)        
        print(_)
        xt = pd.DataFrame(xt)
        
        return _, pd.DataFrame(xt).set_index(x.index)
    else:
        print("The null hypothesis cannot be rejected")    
        return 1, pd.DataFrame(x)

def inverse_boxcox (data, lambdas):
    power = PowerTransformer(method='yeo-johnson')
    power.lambdas_ = lambdas.values
    return(power.inverse_transform([data]))
    #return inv_boxcox(data, lambdas.values)
    
def transform_boxcox_l(data, l_):
    transformed = pd.DataFrame()

    for i in range(0,len(data.columns)):
        #print(i)
        if l_.iloc[i].values == 1:
            inner_scale = data.iloc[:,i]            
        else:
            inner_scale = pd.DataFrame(stats.yeojohnson((data.iloc[:,i]), lmbda=l_.iloc[i].values))
            
        inner_scale.index = data.index
        transformed = pd.concat([transformed,inner_scale],axis=1)
        
    transformed.columns = data.columns
    return transformed

def transform_boxcox (data):
    transformed = pd.DataFrame()
    transformed_lambdas = pd.DataFrame()

    for i in range(0,len(data.columns)):
        l, inner_scale = testNormal(data.iloc[:,i])
        inner_scale.set_index(data.index)

        transformed_lambdas = pd.concat([transformed_lambdas,pd.DataFrame(pd.Series(l))],axis=0)
        transformed = pd.concat([transformed,inner_scale],axis=1)
        
    transformed.columns = data.columns
    return transformed, transformed_lambdas

def inverse_yeo(og, data_, lambda_):
    values = []
    for i in range(0,len(og)):
        X = og[i]
        X_trans = data_[i]
        if X >= 0 and lambda_ == 0:
            X = exp(X_trans) - 1
        elif X >= 0 and lambda_ != 0:
            X = (X_trans * lambda_ + 1) ** (1 / lambda_) - 1
        elif X < 0 and lambda_ != 2:
            X = 1 - (-(2 - lambda_) * X_trans + 1) ** (1 / (2 - lambda_))
        elif X < 0 and lambda_ == 2:
            X = 1 - exp(-X_trans)
        
        values.append(X)
    return(pd.DataFrame(values))


def revert_yeo (og, data_, lambdas):
    reverted = pd.DataFrame()

    for i in range(0,len(data_.columns)):        
        if lambdas.iloc[i].values == 1 :
            revert = data_.iloc[:,i]
        else:
            p#ower = PowerTransformer(method='yeo-johnson')
            #power.lambdas_ = lambdas.iloc[i].values
            #revert = pd.DataFrame(power.inverse_transform([data.iloc[:,i].values]))
            #return inv_boxcox(data, lambdas.values)
            revert = pd.DataFrame(inverse_yeo(og.iloc[:,i].values,data_.iloc[:,i].values, lambdas.iloc[i].values))            
        revert.index = data_.index
        reverted = pd.concat([reverted,revert],axis=1)
        
    reverted.columns = data_.columns
    return reverted

class ZCA(BaseEstimator, TransformerMixin):
  def __init__(self, regularization=1e-5, copy=False):
      self.regularization = regularization
      self.copy = copy
  def fit(self, X, y=None):
      X = as_float_array(X, copy=self.copy)
      self.mean_ = np.mean(X, axis=0)
      X = X - self.mean_
      sigma = np.dot(X.T, X) / (X.shape[0] - 1)
      U, S, V = np.linalg.svd(sigma)
      tmp = np.dot(U, np.diag(1 / np.sqrt(S + self.regularization)))
      self.components_ = np.dot(tmp, U.T)
      return self
  def transform(self, X):
      X_transformed = X - self.mean_
      X_transformed = np.dot(X_transformed, self.components_.T)
      return X_transformed  

'''
def nv_diff_sets(v_of_i,dataset,f_casts):

  s_=sndif_[which(colnames(raw)==var_of_int)]
  d_=ndif_[which(colnames(raw)==var_of_int)]
  
  startRow = c()
  for (r in rownames(dataset[1:d_,,drop=FALSE])):
    startRow = c(startRow,which(rownames(raw)==r))
  
  data_ = c(na.omit(c(dataset[,var_of_int], f_casts)))
  
  if(s_==0):
    inv_d = diffinv(data_,differences=d_,xi=raw[startRow,var_of_int])
  else:  
    inv_d = diffinv(diffinv(data_,differences = d_, xi=raw[startRow,var_of_int]), differences = s_,xi=raw[startRow:(startRow+season-1),var_of_int])
    
  return(inv_d)
'''


def lagpad(x, k):
    length=np.full(abs(k), np.NaN)
    #print(length)
    #k=k-1
    if (k>0):
        result = np.concatenate([length,x[0:(len(x)-k)]])
    elif (k<0):
        result= np.concatenate([(x[abs(k):(len(x))]),length])
    else:
        result= x
    return(result)
   

def lag(data):
    return lagpad(data,1)

array([nan,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.])

In [ ]:
zca = ZCA()

In [ ]:
raw = pd.read_csv("all_data.csv",index_col=0)
raw.index = pd.to_datetime(raw.index)

#raw = dd.read_csv("all_data.csv").set_index('Unnamed: 0')

#fillna(method='bfill')
raw_int = raw.interpolate(method='time').dropna()

#raw_int.dropna(inplace=True)
#.fillna(method='bfill')

In [288]:
raw.diff()

,ASPUS,AWHAETP,BAA10Y,BACDINA066MNFRBNY,BACTSAMFRBDAL,BAMLC0A0CM,BAMLCC0A1AAATRIV,BAMLCC0A4BBBTRIV,BAMLH0A3HYC,BAMLHYH0A0HYM2TRIV,...,WBA,WBEV,WDH,WLTW,WMT,XBOC,XLC,XLF,ZCZZT,^GSPC
2007-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-09-30,-8900.0,0.000000,0.251496,-6.833333,-12.500000,0.385516,4.674825,0.046577,1.944735,-14.288745,...,1.320705,-8.983860,-0.796712,2.232262,0.630589,-2.915589,-4.846204,0.189200,NaN,-0.965743
2007-12-31,4600.0,-0.033333,0.343479,-7.433333,-12.066667,0.412746,13.865394,9.842455,1.360823,7.779844,...,1.313531,2.983182,-0.707303,-4.529913,5.354612,-0.201006,-1.063781,-0.158243,NaN,-1.925058
2008-03-31,-15400.0,0.066667,0.844413,-11.366667,-13.966667,0.740991,11.810032,1.946571,3.595375,-17.523026,...,1.284785,-11.993555,-1.495413,-0.486347,-1.889811,-1.165493,-4.544518,-1.081574,-0.637784,-2.676413
2008-06-30,13800.0,-0.066667,0.024518,-14.733333,3.666667,0.133384,-3.353608,-2.472511,-0.207713,17.155606,...,0.568282,-15.082168,0.109086,-0.058296,2.696235,0.609537,3.069842,0.074054,1.983092,-0.227638
2008-09-30,-19100.0,-0.100000,0.235625,0.900000,-3.066667,0.643051,-2.293021,-4.750389,2.213702,-22.300780,...,0.533210,-3.471185,-0.738303,-3.242127,2.216102,0.333459,-3.060260,-0.193662,0.923125,-1.497190
2008-12-31,-8500.0,-0.200000,2.237218,-18.866667,-31.233333,2.784677,-11.808494,-50.098550,17.539328,-139.901796,...,0.764248,-15.627631,-2.464728,-8.917886,-2.217920,-3.829799,-14.101052,1.069681,-4.677188,-7.416623
2009-03-31,-19600.0,-0.133333,-0.092443,-2.533333,0.133333,-0.425641,8.393091,18.504725,0.678077,19.207534,...,0.174534,-15.750818,-0.429942,-2.932146,-0.577858,-1.805833,-5.209364,-1.055383,-0.375751,-2.172085
2009-06-30,16400.0,-0.166667,-0.836302,31.633333,28.933333,-1.227846,-4.897231,28.017231,-9.396154,73.589231,...,0.080439,0.184815,0.535429,1.851527,0.286145,1.425976,2.434093,0.285505,0.254424,2.336909
2009-09-30,700.0,0.066667,-1.504410,12.400000,17.166667,-1.678937,17.169897,51.165923,-7.782984,77.855357,...,0.062497,3.645675,0.562730,3.206989,0.391863,2.484815,4.163423,0.264454,0.376014,3.040497


In [ ]:
#delta = (raw_int-raw_int.shift()).dropna()
raw_delta = (raw_int - raw_int.apply(lag,0)).dropna()
raw_delta.head()

raw_delta.isna().sum().sum()

In [ ]:
from sklearn.linear_model import LinearRegression

def train(partition):
    est = LinearRegression()
    est.fit(partition[['x']].values, partition.y.values)
    return est

In [ ]:

scaler = StandardScaler()

target = all_data.columns[0]



scaler.fit(np.array(all_data[target]).reshape(-1, 1))

num_folds = 10
kfold = KFold(n_splits=num_folds, shuffle=True)
kfold.get_n_splits(all_data.index)

exclude = 'States'

sig_table = np.zeros(shape=(len(all_data.columns)))
signs_table = np.zeros(shape=(len(all_data.columns)))

p_threshold = .05

New_Names = all_data.columns[2:]
iteration = 0
for train_index, test_index in kfold.split(all_data):
    #print(iteration)
    max_pvalue = 1
    
    subset = all_data.iloc[train_index].loc[:, ~all_data.columns.isin([exclude])]
    
    #skip y and states
    set_ = subset.loc[:, ~subset.columns.isin([target])].columns.tolist()
    
    n=len(subset)
    
    while(max_pvalue>=.05):

        dist = scipy.stats.beta(n/2 - 1, n/2 - 1, loc=-1, scale=2)
        p_values = pd.DataFrame(2*dist.cdf(-abs(subset.pcorr()[target]))).T
        p_values.columns = list(subset.columns)
        
        max_pname = p_values.idxmax(axis=1)[0]
        max_pvalue = p_values[max_pname].values[0]
        
        if (max_pvalue > .05):

            set_.remove(max_pname)
            temp = [target]
            temp.extend(set_)
            subset = subset[temp]
    
    winners = p_values.loc[:, ~p_values.columns.isin([target])].columns.tolist()
    sig_table = (sig_table + np.where(all_data.columns.isin(winners),1,0)).copy()
    
    signs_table[all_data.columns.get_indexer(winners)]+=np.where(subset.pcorr()[target][winners]<0,-1,1)


significance = pd.DataFrame(sig_table).T
significance.columns = list(all_data.columns)
display(significance)

sign = pd.DataFrame(signs_table).T
sign.columns = list(all_data.columns)
display(sign)

purity = abs((sign/num_folds)*(sign/significance)).T.replace([np.inf, -np.inf, np.NaN], 0)
display(purity.T)

threshold = .5

chosen = list(purity.T.columns.values[np.array(purity.T>=threshold).reshape(len(all_data.columns,))])
dataSet = pd.concat([all_data[target],all_data[chosen]],axis=1)
y_scaled = pd.DataFrame(scaler.transform(np.array(dataSet[target]).reshape(-1, 1)))
y_scaled.columns=[target]

display(chosen)

zca_data = pd.concat([y_scaled,pd.DataFrame(zca.fit_transform(dataSet[chosen]),columns=chosen)],axis=1)

zca_data.pcorr()

sns.pairplot(zca_data)


#model = sklearn.linear_model.LinearRegression()
data_set_wConstant = statsmodels.tools.tools.add_constant(zca_data)
y = data_set_wConstant[target]
X = data_set_wConstant[data_set_wConstant.columns.drop(target)]
#results = model.fit(X, y)
model = sm.OLS(y,X)
results = model.fit()

results.summary()

pd.DataFrame(results.get_influence().resid_studentized_internal).hist()

plt.scatter(results.fittedvalues*scaler.scale_[0] + scaler.mean_[0], y*scaler.scale_[0] + scaler.mean_[0])